In [78]:
import os
import sys
import statistics

aoc_year, aoc_day = os.getcwd().split(os.sep)[-2:]

# Download today puzzle & input
!aoc --version
!aoc download -i input.txt --overwrite -p README.md --year {aoc_year} --day {aoc_day}

aoc-cli 0.6.0
Loaded session cookie from "/home/kev/.adventofcode.session".
Fetching puzzle for day 8, 2022...
Saving puzzle description to "README.md"...
Saving puzzle input to "input.txt"...
Done!


\--- Day 8: Treetop Tree House ---
----------

The expedition comes across a peculiar patch of tall trees all planted carefully in a grid. The Elves explain that a previous expedition planted these trees as a reforestation effort. Now, they're curious if this would be a good location for a [tree house](https://en.wikipedia.org/wiki/Tree_house).

First, determine whether there is enough tree cover here to keep a tree house *hidden*. To do this, you need to count the number of trees that are *visible from outside the grid* when looking directly along a row or column.

The Elves have already launched a [quadcopter](https://en.wikipedia.org/wiki/Quadcopter) to generate a map with the height of each tree (your puzzle input). For example:

```
30373
25512
65332
33549
35390

```

Each tree is represented as a single digit whose value is its height, where `0` is the shortest and `9` is the tallest.

A tree is *visible* if all of the other trees between it and an edge of the grid are *shorter* than it. Only consider trees in the same row or column; that is, only look up, down, left, or right from any given tree.

All of the trees around the edge of the grid are *visible* - since they are already on the edge, there are no trees to block the view. In this example, that only leaves the *interior nine trees* to consider:

* The top-left `5` is *visible* from the left and top. (It isn't visible from the right or bottom since other trees of height `5` are in the way.)
* The top-middle `5` is *visible* from the top and right.
* The top-right `1` is not visible from any direction; for it to be visible, there would need to only be trees of height *0* between it and an edge.
* The left-middle `5` is *visible*, but only from the right.
* The center `3` is not visible from any direction; for it to be visible, there would need to be only trees of at most height `2` between it and an edge.
* The right-middle `3` is *visible* from the right.
* In the bottom row, the middle `5` is *visible*, but the `3` and `4` are not.

With 16 trees visible on the edge and another 5 visible in the interior, a total of `*21*` trees are visible in this arrangement.

Consider your map; *how many trees are visible from outside the grid?*


In [79]:
from pprint import pprint

with open('input.txt', 'rt') as f:
    lines = [x.strip() for x in f.readlines()]

# Verify parse
pprint(lines[:6])

['100021112110202312022010330204312040000111012143445142221414240220240442332040010320133120230011020',
 '111110201332323210211143123214321343332124211413115514155115511033421001222101204330001300333011010',
 '222100123300231230122203432310224441551434231352112532354252244334410042212441233243220102033110020',
 '212210311310102334321243104133012535144151532555155341325352512544453402400411340202133231000102020',
 '112002102322001003304233302040124544411225523533534235113353522233535550432232202401221022110311110',
 '112102131331201432320312233124434232544144233241123334112232531521542551332434224211234133132330300']


In [80]:
plantation = [[int(i) for i in list(row)] for row in lines]
#plantation

In [81]:
def is_visible(plantation, row, col):
    height = plantation[row][col]

    # Line-Of-Sight from left 
    los = plantation[row][:col]
    if height > max(los): return True

    # From right
    los = plantation[row][col+1:]
    if height > max(los): return True

    # From top
    los = [row[col] for row in plantation[:row]]
    if height > max(los): return True
    
    # From bot
    los = [row[col] for row in plantation[row+1:]]
    if height > max(los): return True

    #print(f"[{row+1}/{len(plantation)},{col+1}/{len(plantation[row])}] {height} --> {los}")
    return False

interior = 0
for row, trees in enumerate(plantation):
    if row == 0 or row == len(plantation) - 1: continue
    for col, tree in enumerate(trees):
        if col == 0 or col == len(trees) - 1: continue
        if is_visible(plantation, row, col):
            interior += 1

border = len(plantation) * 2 + (len(plantation[0]) - 2) * 2
answer1 = interior + border

In [82]:
print("answer 1:", answer1)

answer 1: 1796


----

In [83]:
# Download part 2
!aoc download --description-only --overwrite --puzzle-file README.md --year {aoc_year} --day {aoc_day}

Loaded session cookie from "/home/kev/.adventofcode.session".
Fetching puzzle for day 8, 2022...
Saving puzzle description to "README.md"...
Done!


\--- Part Two ---
----------

Content with the amount of tree cover available, the Elves just need to know the best spot to build their tree house: they would like to be able to see a lot of *trees*.

To measure the viewing distance from a given tree, look up, down, left, and right from that tree; stop if you reach an edge or at the first tree that is the same height or taller than the tree under consideration. (If a tree is right on the edge, at least one of its viewing distances will be zero.)

The Elves don't care about distant trees taller than those found by the rules above; the proposed tree house has large [eaves](https://en.wikipedia.org/wiki/Eaves) to keep it dry, so they wouldn't be able to see higher than the tree house anyway.

In the example above, consider the middle `5` in the second row:

```
30373
25512
65332
33549
35390

```

* Looking up, its view is not blocked; it can see `*1*` tree (of height `3`).
* Looking left, its view is blocked immediately; it can see only `*1*` tree (of height `5`, right next to it).
* Looking right, its view is not blocked; it can see `*2*` trees.
* Looking down, its view is blocked eventually; it can see `*2*` trees (one of height `3`, then the tree of height `5` that blocks its view).

A tree's *scenic score* is found by *multiplying together* its viewing distance in each of the four directions. For this tree, this is `*4*` (found by multiplying `1 * 1 * 2 * 2`).

However, you can do even better: consider the tree of height `5` in the middle of the fourth row:

```
30373
25512
65332
33549
35390

```

* Looking up, its view is blocked at `*2*` trees (by another tree with a height of `5`).
* Looking left, its view is not blocked; it can see `*2*` trees.
* Looking down, its view is also not blocked; it can see `*1*` tree.
* Looking right, its view is blocked at `*2*` trees (by a massive tree of height `9`).

This tree's scenic score is `*8*` (`2 * 2 * 1 * 2`); this is the ideal spot for the tree house.

Consider each tree on your map. *What is the highest scenic score possible for any tree?*


In [84]:
def scenic_score(plantation, row, col):
    """Brute force approach to search"""
    height = plantation[row][col]

    left = 0
    for i in range(col-1, -1, -1):
        left += 1
        if plantation[row][i] >= height: break

    right = 0
    for i in range(col+1, len(plantation[row])):
        right += 1
        if plantation[row][i] >= height: break

    up = 0
    for i in range(row-1, -1, -1):
        up += 1
        if plantation[i][col] >= height: break

    down = 0
    for i in range(row+1, len(plantation)):
        down += 1
        if plantation[i][col] >= height: break

    return left * right * up * down, [up, left, down, right]

max_score = 0
for row, trees in enumerate(plantation):
    # Tree on edge will have total score 0
    if row == 0 or row == len(plantation) - 1: continue
    for col, tree in enumerate(trees):
        # Tree on edge will have total score 0
        if col == 0 or col == len(trees) - 1: continue
        score, dims = scenic_score(plantation, row, col)
        if score > max_score:
            max_score = score
            print(f"[{row},{col}] {dims} = {score}")

answer2 = max_score

[1,1] [1, 1, 1, 1] = 1
[1,4] [1, 1, 2, 2] = 4
[1,6] [1, 6, 2, 3] = 36
[1,14] [1, 2, 5, 8] = 80
[1,22] [1, 22, 2, 7] = 308
[1,80] [1, 14, 4, 18] = 1008
[2,24] [2, 24, 5, 8] = 1920
[3,67] [3, 4, 10, 31] = 3720
[4,20] [4, 20, 7, 8] = 4480
[5,15] [5, 15, 8, 14] = 8400
[5,36] [3, 36, 8, 24] = 20736
[9,60] [9, 11, 7, 38] = 26334
[14,52] [14, 52, 2, 46] = 66976
[29,46] [29, 46, 16, 4] = 85376
[48,23] [48, 23, 9, 9] = 89424
[63,34] [7, 34, 35, 14] = 116620
[75,62] [8, 22, 23, 36] = 145728
[86,49] [10, 49, 12, 49] = 288120


In [85]:
print("answer 2:", answer2)

answer 2: 288120
